In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
def quizizz_manual_grading(quizizz_student, quizizz_correct_ans, 
                           colname_correct_ans, Nquestion = 999, 
                           outputfile="realscore.xlsx", 
                           sort = True, wrongscore = True, testtime = True):
    """
    A function to do automate "manual" grading on quizizz
    INPUT:
        a must:
        - quizizz_student = Excel file from quizizz contain the student answer
        - quizizz_correct_ans = Excel file from quizizz contain the correct answer
        - colname_correct_ans = Name of user (column name) to get the correct answer in "quizizz_correct_ans" file 
        - Nquestion = number of question
        optional:
        - outputfile = output file name for correct score [default = realscore.xlsx]
        - sort = True/False, sort name based on score
        - wrongscore = True/Flase, add "wrong score / quizizz score" column
        - testtime = True/False, add "total time" column
    """
    
    stud = pd.read_excel(quizizz_student, sheet_name="Overview")
    student = stud.loc[:Nquestion-1]
    correctans = pd.read_excel(quizizz_correct_ans, sheet_name="Overview").loc[:Nquestion-1]
    
    time_data = pd.read_excel(quizizz_student, sheet_name="Time Data").loc[:Nquestion-1]
    time_name = time_data.columns[5:] # 2 sheet have a bit different name
    
    list_student = student.columns[8:]
    
    new_df = pd.DataFrame()
    list_score = []
    list_totaltime = []
    
    for i, name in enumerate(list_student):
        score = (100./Nquestion) * np.sum(student[name] == correctans[colname_correct_ans])
        list_score.append(score)
        
        time_student = time_data[time_name[i]]
        time_sum = datetime.timedelta()
        for i, x in enumerate(time_student):
            if x != '-':
                tx = datetime.timedelta(minutes=x.minute, seconds=x.second)
                time_sum += tx
        list_totaltime.append(str(time_sum))

    new_df["Name"] = list_student
    new_df["Real Score"] = list_score
    
    if wrongscore:
        new_df["Wrong Score"] = stud.loc[Nquestion][8:].str.rstrip('%').astype('float').values
    
    if testtime:
        new_df["Total Time"] = list_totaltime
    
    
    if sort:
        sorted_df = new_df.sort_values(by=['Real Score'], ascending=False)
        sorted_df.to_excel(outputfile, index=False)
        return sorted_df
    else:
        new_df.to_excel(outputfile, index=False)
        return new_df

In [3]:
quizizz_manual_grading("result_student.xlsx", "correct_answer.xlsx", "Sulis (Sulis ) ", 25, "cheat_me_if_you_can.xlsx")

,Name,Real Score,Wrong Score,Total Time
84,Yasmin 120290022 (Yasmin 120290022),68.0,12.0,1:05:30
82,Muhammad Rizki Kurniawan (Muhammad Rizki Kurni...,64.0,16.0,1:14:40
77,Wildan Muharram (Wildan Muharram),60.0,20.0,1:08:47
83,Praz_ 120290031 (Praz_ 120290031),56.0,12.0,1:15:41
72,RAYHAN RIZKI H_120290086 (RAYHAN HARYONO),48.0,24.0,1:02:37
...,...,...,...,...
34,12O29OO4O_Elsa Claudia (12O29OO4O Sidabutar),0.0,84.0,1:10:46
35,Flowerine Olvie_120290045 (Flowerine Olvie),0.0,92.0,1:26:00
38,Fahira R_120290085 (12O29OO85_Fahira Ramadhian...,0.0,80.0,1:05:37
39,Maulida Aqsha_120290066 (Maulida Aqsha_1202900...,0.0,80.0,1:17:43
